# Movie Recommendation System using Doc2vec Embeddings and Vector DB

This Colab notebook aims to illustrate the process of creating a recommendation system using embeddings and a Vector DB.

This approach involves combining the various movie genres or characteristics of a movie to form Doc2Vec embeddings, which offer a comprehensive portrayal of the movie content.

These embeddings serve dual purposes: they can either be directly inputted into a classification model for genre classification or stored in a VectorDB. By storing embeddings in a VectorDB, efficient retrieval and query search for recommendations become possible at a later stage.


### Installing the relevant dependencies


In [ ]:
!pip install torch scikit-learn lancedb nltk gensim lancedb scipy==1.12 kaggle

## Kaggle Configuration and Data Needs

We are using a movies metadata data which is being uploaded on the Kaggle. To download the dataset and use it for our recommendation system, we will need a `kaggle.json` file containing our creds.

You can download the `kaggle.json` file from your Kaggle account settings. Follow these steps and make your life easy.

1. Go to Kaggle and log in to your account.
2. Navigate to Your Account Settings and click on your profile picture in the top right corner of the page, Now From the dropdown menu, select `Account`.
3. Scroll down to the `API` section, Click on `Create New API Token`. This will download a file named kaggle.json to your computer.

Once you have the `kaggle.json` file, you need to upload it here on colab data space. After uploading the `kaggle.json` file, run the following code to set up the credentials and download the dataset in `data` directory

In [ ]:
import json
import os

# Assuming kaggle.json is uploaded to the current directory
with open("kaggle.json") as f:
    kaggle_credentials = json.load(f)

os.environ["KAGGLE_USERNAME"] = kaggle_credentials["username"]
os.environ["KAGGLE_KEY"] = kaggle_credentials["key"]

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize the Kaggle API
api = KaggleApi()
api.authenticate()

# Specify the dataset you want to download
dataset = "rounakbanik/the-movies-dataset"
destination = "data/"

# Create the destination directory if it doesn't exist
if not os.path.exists(destination):
    os.makedirs(destination)

# Download the dataset
api.dataset_download_files(dataset, path=destination, unzip=True)

print(f"Dataset {dataset} downloaded to {destination}")

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Read data from CSV file
movie_data = pd.read_csv(
    "/Users/vipul/Nova/Projects/genre_spectrum/movies_metadata.csv", low_memory=False
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def preprocess_data(movie_data_chunk):
    tagged_docs = []
    valid_indices = []
    movie_info = []

    # Wrap your loop with tqdm
    for i, row in tqdm(movie_data_chunk.iterrows(), total=len(movie_data_chunk)):
        try:
            # Constructing movie text
            movies_text = ""
            movies_text += "Overview: " + row["overview"] + "\n"
            genres = ", ".join([genre["name"] for genre in eval(row["genres"])])
            movies_text += "Overview: " + row["overview"] + "\n"
            movies_text += "Genres: " + genres + "\n"
            movies_text += "Title: " + row["title"] + "\n"
            tagged_docs.append(
                TaggedDocument(words=word_tokenize(movies_text.lower()), tags=[str(i)])
            )
            valid_indices.append(i)
            movie_info.append((row["title"], genres))
        except Exception as e:
            continue

    return tagged_docs, valid_indices, movie_info


def train_doc2vec_model(tagged_data, num_epochs=20):
    # Initialize Doc2Vec model
    doc2vec_model = Doc2Vec(vector_size=100, min_count=2, epochs=num_epochs)
    doc2vec_model.build_vocab(tqdm(tagged_data, desc="Building Vocabulary"))
    for epoch in range(num_epochs):
        doc2vec_model.train(
            tqdm(tagged_data, desc=f"Epoch {epoch+1}"),
            total_examples=doc2vec_model.corpus_count,
            epochs=doc2vec_model.epochs,
        )

    return doc2vec_model


# Preprocess data and extract genres for the first 1000 movies
chunk_size = 1000
tagged_data = []
valid_indices = []
movie_info = []
for chunk_start in range(0, len(movie_data), chunk_size):
    movie_data_chunk = movie_data.iloc[chunk_start : chunk_start + chunk_size]
    chunk_tagged_data, chunk_valid_indices, chunk_movie_info = preprocess_data(
        movie_data_chunk
    )
    tagged_data.extend(chunk_tagged_data)
    valid_indices.extend(chunk_valid_indices)
    movie_info.extend(chunk_movie_info)

doc2vec_model = train_doc2vec_model(tagged_data)

Epoch 20: 100%|██████████| 44506/44506 [00:02<00:00, 20943.08it/s]


### Training a Neural Network for the Genre Classification Task

In [ ]:
# Extract genre labels for the valid indices
genres_list = []
for i in valid_indices:
    row = movie_data.loc[i]
    genres = [genre["name"] for genre in eval(row["genres"])]
    genres_list.append(genres)

mlb = MultiLabelBinarizer()
genre_labels = mlb.fit_transform(genres_list)

embeddings = []
for i in valid_indices:
    embeddings.append(doc2vec_model.dv[str(i)])
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, genre_labels, test_size=0.2, random_state=42
)

X_train_np = np.array(X_train, dtype=np.float32)
y_train_np = np.array(y_train, dtype=np.float32)
X_test_np = np.array(X_test, dtype=np.float32)
y_test_np = np.array(y_test, dtype=np.float32)

X_train_tensor = torch.tensor(X_train_np)
y_train_tensor = torch.tensor(y_train_np)
X_test_tensor = torch.tensor(X_test_np)
y_test_tensor = torch.tensor(y_test_np)


class GenreClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(GenreClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)  # Adjust the dropout rate as needed

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x


# Move model to the selected device
model = GenreClassifier(input_size=100, output_size=len(mlb.classes_)).to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
batch_size = 64

train_dataset = TensorDataset(X_train_tensor.to(device), y_train_tensor.to(device))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}")

### Testing the `model` to see if our model is able to predict the genres for the movies from the test dataset

In [ ]:
from sklearn.metrics import f1_score

model.eval()
with torch.no_grad():
    X_test_tensor, y_test_tensor = X_test_tensor.to(device), y_test_tensor.to(
        device
    )  # Move test data to device
    outputs = model(X_test_tensor)
    test_loss = criterion(outputs, y_test_tensor)
    print(f"Test Loss: {test_loss.item():.4f}")


thresholds = [0.1] * len(mlb.classes_)
thresholds_tensor = torch.tensor(thresholds, device=device).unsqueeze(0)

# Convert the outputs to binary predictions using varying thresholds
predicted_labels = (outputs > thresholds_tensor).cpu().numpy()

# Convert binary predictions and actual labels to multi-label format
predicted_multilabels = mlb.inverse_transform(predicted_labels)
actual_multilabels = mlb.inverse_transform(y_test_np)

# Print the Predicted and Actual Labels for each movie
for i, (predicted, actual) in enumerate(zip(predicted_multilabels, actual_multilabels)):
    print(f"Movie {i+1}:")
    print(f"    Predicted Labels: {predicted}")
    print(f"    Actual Labels: {actual}")


# Compute F1-score
f1 = f1_score(y_test_np, predicted_labels, average="micro")
print(f"F1-score: {f1:.4f}")

# Saving the trained model
torch.save(model.state_dict(), "trained_model.pth")

### Storing the Doc2Vec Embeddings into LanceDB VectorDatabase

In [ ]:
import lancedb
import numpy as np
import pandas as pd

data = []

for i in valid_indices:
    embedding = doc2vec_model.dv[str(i)]
    title, genres = movie_info[valid_indices.index(i)]
    data.append({"title": title, "genres": genres, "vector": embedding.tolist()})

db = lancedb.connect(".db")
tbl = db.create_table("doc2vec_embeddings", data, mode="Overwrite")
db["doc2vec_embeddings"].head()

In [ ]:
def get_recommendations(title):
    pd_data = pd.DataFrame(data)
    result = (
        tbl.search(pd_data[pd_data["title"] == title]["vector"].values[0])
        .metric("cosine")
        .limit(10)
        .to_pandas()
    )
    return result[["title"]]

### D-Day : Let's generate some recommendations

In [ ]:
get_recommendations("Vertical Limit")

,title
0,Vertical Limit
1,Demons of War
2,Fear and Desire
3,Escape from Sobibor
4,Last Girl Standing
5,K2: Siren of the Himalayas
6,Ghost Ship
7,Camp Massacre
8,Captain Nemo and the Underwater City
9,Seas Beneath
